In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("/home/missinguser/CSE/single-stage-opt/hybrid_tokamak/laptop/")
import latexplot
latexplot.set_cmap() 

from simsopt import mhd
from simsopt import geo
from simsopt import objectives
import matplotlib.pyplot as plt
import glob
import py_spec

In [13]:
# Directories with data
directories = {
    "rot_ell_fr": "../../freeb_high_aspect_combined",
    "QA_20_fr": "../../freeb_12-11-16-48-29",
    # "QI_10_fr": "../../freeb_12-23-01-03-52", # Needs different QuasisymmetryRatioResidual, since m=1, n=1
    
    "rot_ell": "../../fixb_12-06-11-52-22",
    "QA_20": "../../fixb_12-20-00-39-00",
}


def get_results(surf, res):
    # Vmec setup
    vmec = mhd.Vmec(verbose=False)
    vmec.boundary = surf
    qs = mhd.QuasisymmetryRatioResidual(vmec, surfaces=np.linspace(0.1, 1, 16), helicity_m=1, helicity_n=0, ntheta=32, nphi=32)

    # Compute quantities
    mean_iota_diff = (vmec.mean_iota() - 0.4384346834911653) ** 2
    f_qs = qs.total()
    r0_diff = (surf.get_rc(0, 0) - 1.0) ** 2
    return {"Directory": key, "Resolution": res, "$f_{QS}$": f_qs, "$(\iota - \iota^*)^2$": mean_iota_diff, "$(R_0 -R_0^*)^2$": r0_diff}

# Collect results
results = []
for key, directory in directories.items():
    subdirs = sorted(glob.glob(f"{directory}/mpol*"))

    if "freeb" in directory:
        # Freeboundary
        first_file = sorted(glob.glob(f"{subdirs[0]}/*_000_000000.sp.h5"))[0]
        print(first_file)
        spec_output = py_spec.output.SPECout(first_file)
        surf = mhd.Spec.pyspec_to_simsopt_surf(spec_output, 0)
        results.append(get_results(surf, 0))
    else:
        first_file = sorted(glob.glob(f"{subdirs[0]}/input*_000_000000"))[0]
        results.append(get_results( geo.SurfaceRZFourier.from_vmec_input(first_file), 0))
    print(first_file)
    for subdir in subdirs:
        res = int(subdir.split("mpol")[-1])
        if "freeb" in directory:
            latest_file = max(glob.glob(f"{subdir}/*_000_*.h5"), key=lambda x: int(x.split("_")[-1].split(".")[0]))
            print(latest_file)
            if latest_file.endswith(".h5"):
                spec_output = py_spec.output.SPECout(latest_file)
                surf = mhd.Spec.pyspec_to_simsopt_surf(spec_output, 0)

                results.append(get_results(surf, res))
        else:
            # Fixboundary
            latest_file = max(glob.glob(f"{subdir}/input*_000_*"), key=lambda x: int(x.split("_")[-1]))
            print(latest_file)
            results.append(get_results( geo.SurfaceRZFourier.from_vmec_input(latest_file), res))

# Convert to DataFrame
df = pd.DataFrame(results)

# Save as LaTeX table
with open("results_table.tex", "w") as f:
    f.write(df.to_latex(index=False, float_format="%.6f"))



../../freeb_high_aspect_combined/mpol1/rotating_ellipse_fb_low_000_000000.sp.h5
../../freeb_high_aspect_combined/mpol1/rotating_ellipse_fb_low_000_000000.sp.h5
../../freeb_high_aspect_combined/mpol1/rotating_ellipse_fb_low_000_000090.sp.h5
../../freeb_high_aspect_combined/mpol2/rotating_ellipse_fb_low_000_000103.sp.h5
../../freeb_high_aspect_combined/mpol3/rotating_ellipse_fb_low_000_000213.sp.h5
../../freeb_high_aspect_combined/mpol4/rotating_ellipse_fb_low_000_000213.sp.h5
../../freeb_high_aspect_combined/mpol5/rotating_ellipse_fb_low_000_000218.sp.h5
../../freeb_12-11-16-48-29/mpol1/rotating_ellipse_fb_low_000_000000.sp.h5
../../freeb_12-11-16-48-29/mpol1/rotating_ellipse_fb_low_000_000000.sp.h5
../../freeb_12-11-16-48-29/mpol1/rotating_ellipse_fb_low_000_000000.sp.h5
../../freeb_12-11-16-48-29/mpol2/rotating_ellipse_fb_low_000_000115.sp.h5
../../freeb_12-11-16-48-29/mpol3/rotating_ellipse_fb_low_000_000532.sp.h5
../../freeb_12-11-16-48-29/mpol4/rotating_ellipse_fb_low_000_001141.sp

In [16]:

# Plotting f_{QS} vs Resolution
plot_keys = ["$f_{QS}$", "$(\iota - \iota^*)^2$","$(R_0 -R_0^*)^2$"]
latexplot.figure()
for i,key in enumerate(plot_keys):
    plt.subplot(1, len(plot_keys), i+1)
    for directory in df["Directory"].unique():
        subdf = df[df["Directory"] == directory].sort_values(by="Resolution")
        plt.semilogy(subdf["Resolution"], subdf[key], label=directory, marker="o")  # Add marker for distinction
    plt.xlabel("Resolution")
    plt.ylabel(key)
plt.legend()
latexplot.savenshow("f_qs_vs_resolution")

/home/missinguser/CSE/single-stage-opt/hybrid_tokamak/laptop/latexplot.py:88: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [11]:

# Save as LaTeX table
with open("results_table.tex", "w") as f:
    f.write(df.rename({
        "Directory": "Optimization run",
    }, axis=1).to_latex(index=False, float_format="%.6f"))

In [12]:
df.rename({
        "Directory": "Optimization run",
}, axis=1)

,Optimization run,Resolution,$f_{QS}$,$(\iota - \iota^*)^2$,$(R_0 -R_0^*)^2$
0,rot_ell_fr,0,0.018509,5.916172e-03,4.658199e-07
1,rot_ell_fr,1,0.057365,1.631875e-03,2.630895e-04
2,rot_ell_fr,2,0.010300,2.191473e-03,1.104380e-03
3,rot_ell_fr,3,0.002412,2.165977e-04,1.050582e-03
4,rot_ell_fr,4,0.002412,2.165977e-04,1.050582e-03
5,rot_ell_fr,5,0.000942,4.423836e-04,1.867052e-04
6,QA_20_fr,0,0.039881,4.330685e-03,1.423295e-06
7,QA_20_fr,1,0.039881,4.330685e-03,1.423295e-06
8,QA_20_fr,2,0.591921,9.311796e-03,1.501010e-05
9,QA_20_fr,3,0.005289,2.515644e-03,1.606908e-04
